In [1]:
import cv2
import pdf2image
from PIL import Image
import numpy as np
from pytesseract import pytesseract
import matplotlib as mpl
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
import re

mpl.rcParams['image.cmap'] = 'gray'

path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.tesseract_cmd = path_to_tesseract

In [2]:
start_pdf = 101
end_pdf = 101

In [3]:
def convert_pdf_to_image(file, dpi):
    images_list = []
    images = pdf2image.convert_from_path(file, dpi = dpi, poppler_path = r"C:\Users\Amandeep Singh\Downloads\Release-23.05.0-0\poppler-23.05.0\Library\bin")
    
    for i, image in enumerate(images):
        image_np = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        images_list.append(image_np)
        
    return images_list

In [4]:
def f(x):
    global horizontal_lines_img, verticle_lines_img, kernel
    kernel_length = np.array(img).shape[1]//x

    verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
    hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    img_temp1 = cv2.erode(img_bin, verticle_kernel, iterations=3)
    verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
    
    cv2.imwrite("Images/verticle_lines.jpg",verticle_lines_img)
    img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
    horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)

In [5]:
def sort_contours(cnts, method):
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),key=lambda b:[b[1][1],b[1][0]] , reverse=False))
    return (cnts, boundingBoxes)

In [26]:
for pdf_file in range(start_pdf, end_pdf+1):
    images = convert_pdf_to_image(str(pdf_file)+".pdf", 300)
    
    final_list = []
    count = 0 
    for i in range(2, len(images)-1):
        img = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
        
        (thresh, img_bin) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        img_bin = 255 - img_bin
        cv2.imwrite("Images/Image_bin.jpg", img_bin)

        interact(f, x = widgets.IntSlider(min = 1, max = 255, step = 1, value = 50));
        
        alpha = 0.5
        beta = 1.0 - alpha
        img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
        img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
        (thresh, img_final_bin) = cv2.threshold(img_final_bin, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        
        contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        (contours, boundingBoxes) = sort_contours(contours,"top-to-bottom")
        
        g = []
        for c in contours:
            x, y, w, h = cv2.boundingRect(c)
            if(w < 800 and w > 700 and h > 320):
                g.append([x, y, w, h])
                
        a = []
        for i in range(len(g)):
            if i == 0:
                a.append(g[i])
            else:
                if np.abs(g[i][0] - g[i-1][0]) < 50:
                    pass
                else:
                    a.append(g[i])
               
        for i in a:
            new_img = img[i[1]:i[1]+i[3], i[0]:i[0]+i[2]]
            dic = {}
            
            extracted_text = pytesseract.image_to_string(new_img, lang='guj', config='--oem 3 --psm 6')
            extracted_text = extracted_text.split()
            print(extracted_text)
            print("---")
            
            #Getting name from the list
            start_index_for_name = None
            end_index_for_name = None
            for i, item in enumerate(extracted_text):
                if item.endswith("દારનું") or item.endswith("દારનું") or item.startswith("મત્\u200cદારનું"):
                    start_index_for_name = i
                elif (item.startswith("પિતાનું") or item.startswith("પતિનું")) and end_index_for_name is None:
                    end_index_for_name = i
                    break
            
            dic["Name"] = "Start index is None" if start_index_for_name is None else None
            dic["Name"] = "End index is None" if end_index_for_name is None else None
            
            if start_index_for_name is not None and end_index_for_name is not None:
                name = " ".join(extracted_text[start_index_for_name:end_index_for_name])
                
                pattern = r'[:;]'
                name = re.split(pattern, name, maxsplit = 1)[-1].strip()
                
                dic["Name"] = name
            else:
                dic["Name"] = "None"
                
            #Getting relation
            start_index_for_relation = None
            end_index_for_relation = None
            relation = None
            for i, item in enumerate(extracted_text):
                if item.startswith("પિતાનું") and start_index_for_relation is None:
                    relation = "Father"
                    start_index_for_relation = i
                    break
                elif item.startswith("પતિનું") and start_index_for_relation is None:
                    relation = "Husband"
                    start_index_for_relation = i
                    break
            
            for i, item in enumerate(extracted_text):
                if (item.startswith('ઘર') or item.startswith('ઘર')) and end_index_for_relation is None:
                    end_index_for_relation = i
                    break
                    
            if start_index_for_relation is not None and end_index_for_relation is not None:
                    relation_name = " ".join(extracted_text[start_index_for_relation:end_index_for_relation])
                    
                    pattern = r'[:;]'
                    relation_name = re.split(pattern, relation_name, maxsplit = 1)[-1].strip()
                    relation_name = relation_name.split()
                    relation_name = ' '.join(relation_name[:2])
                    
            if relation == "Father":
                dic["FatherName"] = relation_name
            else:
                dic["FatherName"] = ""
                
            if relation == "Husband":
                dic["HusbandName"] = relation_name
            else:
                dic["HusbandName"] = ""
                
            dic["num"] = count
            
            print(dic)
            final_list.append(dic)
            print("===")
            count += 1
            
    sd = pd.DataFrame(final_list)
    sd[["Name", "FatherName", "HusbandName"]].to_excel(str(pdf_file) + ".xlsx")
    
    print(f"{pdf_file} completed")
    
print("===")
print("end")

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

['સા', '12342780', 'મતદારનું', 'નામ', ':', 'રાજેશચંદ્ર', 'સકસેના', 'પિતાનું', 'નામ:', 'અર્જુનસિંઘ', 'સકસેના', 'ઘર', 'નંબર', ':', '5-અ', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '59', 'જાતિ', ':', 'પુરુષ']
---
{'Name': 'રાજેશચંદ્ર સકસેના', 'FatherName': 'અર્જુનસિંઘ સકસેના', 'HusbandName': '', 'num': 0}
===
['|', 'થુ', '(12342798', 'મતદારનું', 'નામ', ':', 'કંચનબેન', 'સકસેના', 'પતિનું', 'નાંમ:', 'રાજેશચંદ્ર', 'સકસેના', 'જિમિ', 'ઘર', 'નૅબર', ':', '5-અ', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '47', 'જાતિ', ':', 'સ્ત્રી']
---
{'Name': 'કંચનબેન સકસેના', 'FatherName': '', 'HusbandName': 'રાજેશચંદ્ર સકસેના', 'num': 1}
===
['[|]', '3]', 'ઉ]/04/018/201610', 'મૃતદારનું', 'નામ', ':', 'દીલીપભાઇ', 'ચેટરજી', 'પિતાનું', 'નામ:', 'જગબંધુભાઇ', 'ચેટરજી', 'મે', 'ઘર', 'નંબર', ':', '7', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '59', 'જાતિ', ':', 'પુરુષ']
---
{'Name': 'દીલીપભાઇ ચેટરજી', 'FatherName': 'જગબંધુભાઇ ચેટરજી', 'HusbandName': '', 'num': 2}
===
['[|', '41', 'ઉ]/04/018/201611', 'મતદારનું', 'નામ', ':', 'રીનાબેન', 'ચેટરજી', 'પતિનું', '

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

['મતદારનું', 'નામ', ':', 'સરોજબેન', 'મીયાવડા', 'પતિનું', 'નાંમ:', 'ભરતભાઇ', 'મીયાવડા', 'ઘર', 'નેંબર', ':', '37', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '44', 'જાતિ', ':', 'સ્ત્રી']
---
{'Name': 'સરોજબેન મીયાવડા', 'FatherName': '', 'HusbandName': 'ભરતભાઇ મીયાવડા', 'num': 30}
===
['મૃતદારનું', 'નામ', ';', 'બહાડરમાઇ', 'મીયાવડા', 'પિતાનું', 'નામ:', 'ધીરૂભાઈ', 'મીયાવડા', 'મિ', 'ઘર', 'નંબર', ':', '37', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '42', 'જાતિ', ':', 'પુરુષ']
---
{'Name': 'બહાડરમાઇ મીયાવડા', 'FatherName': 'ધીરૂભાઈ મીયાવડા', 'HusbandName': '', 'num': 31}
===
['મતદારનું', 'નામ', ':', 'મીનાબેન', 'મીયાવડા', 'પતિનું', 'નામ:', 'બહાદુરભાઇ', 'મીયાવડા', 'સિ', 'ઘર', 'નૅબર', ':', '37', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '39', 'જાતિ', ':', 'સ્ત્રી']
---
{'Name': 'મીનાબેન મીયાવડા', 'FatherName': '', 'HusbandName': 'બહાદુરભાઇ મીયાવડા', 'num': 32}
===
['|]', '૩]', 'ઊ[/06/042/292103', 'મૃતદારનું', 'નામ', ':', 'લલીતભાઇ', 'બગડા', 'પિતાનું', 'નામ:', 'દેસાભાઇ', 'બગડા', 'ર', 'ઘર', 'નંબર', ':', '40', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', '

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

...

['મૃતદારનું', 'નામ', ':', 'દિનેશભાઈ', 'યાદવ', 'પિતાનું', 'નામ:', 'રામનિવાસભાઈ', 'યાદવ', 'ઘર', 'નંબર', ':', '૧૦૪-બી', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '41', 'જાતિ', ':', 'પુરુષ']
---
{'Name': 'દિનેશભાઈ યાદવ', 'FatherName': 'રામનિવાસભાઈ યાદવ', 'HusbandName': '', 'num': 496}
===
['૪1012785756', 'મતદારનું', 'નામ', ':', 'રાજકુમારી', 'યાદવ', 'પતિનું', 'નામ:', 'દિનેશભાઈ', 'યાદવ', 'ઘર', 'નૅબર', ':', '૧૦૪-બી', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '39', 'જાતિ', ':', 'સ્ત્રી']
---
{'Name': 'રાજકુમારી યાદવ', 'FatherName': '', 'HusbandName': 'દિનેશભાઈ યાદવ', 'num': 497}
===
['૪102489326', 'મૃતદારનું', 'નામ', ':', 'રણછોડ', 'મેર', 'પિતાનું', 'નામ:', 'પ્રભુભાઇ', 'મેર', 'ક', 'ઘર', 'નંબર', ':', '૧રર-બી-ઇ', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '30', 'જાતિ', ':', 'પુરુષ']
---
{'Name': 'રણછોડ મેર', 'FatherName': 'પ્રભુભાઇ મેર', 'HusbandName': '', 'num': 498}
===
['૪013279254', 'મૃતદારનું', 'નામ', ':', 'હાર્દિક', 'ચૌહાણ', 'પિતાનું', 'નામ:', 'અશોકભાઇ', 'ચૌહાણ', '.', 'ઘર', 'નંબર', ':', '198', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '25

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

['ઊ/04/018/072205', 'મૃતદારનું', 'નામ', ':', 'તંગરાજ', 'આદીવીડર', 'પિતાનું', 'નામ:', 'મીજાસ્વામી', 'આદીવીડર', 'ઘર', 'નંબર', ':', '296', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '65', 'જાતિ', ':', 'પુરુષ']
---
{'Name': 'તંગરાજ આદીવીડર', 'FatherName': 'મીજાસ્વામી આદીવીડર', 'HusbandName': '', 'num': 526}
===
['લ/04/018/072137', 'મતદારનું', 'નામ', ':', 'સરોજા', 'આદીવીડર', 'પતિનું', 'નામ:', 'તંગરાજ', 'આદીવીડર', 'ઘર', 'નબર', ':', '296', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '63', 'જાતિ', ':', 'સ્ત્રી']
---
{'Name': 'સરોજા આદીવીડર', 'FatherName': '', 'HusbandName': 'તંગરાજ આદીવીડર', 'num': 527}
===
['મૃતદારનું', 'નામ', ':', 'હરીકૃષ્ણન', 'આદીવીડર', 'પિતાનું', 'નામ:', 'તંગરાજ', 'આદીવીડર', 'બિ', 'ઘર', 'નંબર', ':', '296', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '36', 'જાતિ', ':', 'પુરુષ']
---
{'Name': 'હરીકૃષ્ણન આદીવીડર', 'FatherName': 'તંગરાજ આદીવીડર', 'HusbandName': '', 'num': 528}
===
['ઊ।/04/018/072135', 'મતદારનું', 'નામ', ':', 'રાજમણીકમ', 'આદીવેડર', 'ર', 'પિતાનું', 'નામ:', 'મુરતન', 'આદીવીડર', 'ફોટો', 'ઉપલબ્ધ', 'ઘર', 'નબ

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

['રા', '1571620822', 'મતદારનું', 'નામ', ':', 'વસંતભાઇ', 'સોનગ્રા', 'પિતાનું', 'નામ:', 'છગનભાઇ', 'સોનગ્રા', 'ઘર', 'નંબર', ':', 'ત્ય્પે-3-8/બી', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '54', 'જાતિ', ':', 'પુરુષ']
---
{'Name': 'વસંતભાઇ સોનગ્રા', 'FatherName': 'છગનભાઇ સોનગ્રા', 'HusbandName': '', 'num': 535}
===
['|', '2', '1571620764', 'મતદારનું', 'નામ', ':', 'શારદાબેન', 'સોનગ્રા', 'પતિનું', 'નામ:', 'વસંતભાઇ', 'સોનગ્રા', 'મિ', 'ઘર', 'નબર', ':', 'ટાઈપ', '-3', '-', '8/બી', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '54', 'જાતિ', ':', 'સ્ત્રી']
---
{'Name': 'શારદાબેન સોનગ્રા', 'FatherName': '', 'HusbandName': 'વસંતભાઇ સોનગ્રા', 'num': 536}
===
['દા', '22145092', 'મતદારનું', 'નામ', ':', 'દેવેન્દ્ર', 'સોનગરા', 'પિતાનું', 'નામ:', 'વસંતભાઇ', 'સોનાગ્રા', 'પિ', 'ઘર', 'નંબર', ':', 'ટાઈપ', '3', '-', '8/બી', 'ફોટો', 'ઉપલબ્ધ', 'ઉમર', ':', '26', 'જાતિ', ':', 'પુરુષ']
---
{'Name': 'દેવેન્દ્ર સોનગરા', 'FatherName': 'વસંતભાઇ સોનાગ્રા', 'HusbandName': '', 'num': 537}
===
101 completed
===
end
